### Exercise 1

Let $\theta_1$ and $\theta_2$ be real valued parameters of the model

$$
y = \frac{\theta_1 x}{\theta_2 + x}.
$$

a. Choose two suitable prior distributions for $\theta_1$ and $\theta_2$  and use HMC algorithm to find their posterior distributions, conditioning on the observations

$$
x=(28,55,110,138,225,375)\\
y=(0.053,0.060,0.112,0.105,0.099,0.122).
$$

b. Discuss how different parameters for both priors and the HMC algorithm lead to different estimates. 

c. Plot the most reliable posterior distributions, according to convergence checks on the traces.

In [95]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pyro
import pyro.distributions as dist
from pyro.infer.mcmc import MCMC, HMC, NUTS

In [96]:
def model(obs_x = torch.tensor([(28,55,110,138,225,375)])):
    
    # Suitable prior distribution with mean 0 and variance 1
    theta1 = pyro.sample("theta1", dist.Normal(0,1)) 
    theta2 = pyro.sample("theta2", dist.Normal(0,1))
    
    # Calculated y variable
    y = pyro.param("y", (theta1*obs_x)/ (theta2+obs_x))
    
    return y

In [97]:
def conditioned_model(obs, obs_x = torch.tensor([(28,55,110,138,225,375)])):
    
    # Suitable prior distribution with mean 0 and variance 1
    theta1 = pyro.sample("theta1", dist.Normal(0,1)) 
    theta2 = pyro.sample("theta2", dist.Normal(0,1))
    
    # Calculated y variable
    y = pyro.param("y", (theta1*obs_x)/ (theta2+obs_x), obs = obs)
    
    return y

In [101]:
!ulimit -n 2048

In [102]:
hmc_kernel = HMC(model=conditioned_model) # transition kernel
mcmc = MCMC(hmc_kernel, num_samples=500, warmup_steps=1000, num_chains=3)

# y observations
obs = torch.tensor([0.053,0.060,0.112,0.105,0.099,0.122])

# posteriors 
posterior = mcmc.run(obs = obs)

# dictionary of sampled values
#print(mcmc.get_samples().keys())

mcmc.summary()

Exception in thread Thread-21:
Traceback (most recent call last):
  File "/home/john/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/john/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/john/anaconda3/lib/python3.7/site-packages/pyro/infer/mcmc/mcmc.py", line 28, in logger_thread
    progress_bars = ProgressBar(warmup_steps, num_samples, disable=disable_progbar, num_bars=num_chains)
  File "/home/john/anaconda3/lib/python3.7/site-packages/pyro/infer/mcmc/logger.py", line 69, in __init__
    position=i, file=sys.stderr, disable=disable)
  File "/home/john/anaconda3/lib/python3.7/site-packages/tqdm/notebook.py", line 194, in __init__
    kwargs['bar_format'] = kwargs['bar_format'].replace('{bar}', '<bar/>')
AttributeError: 'NoneType' object has no attribute 'replace'

Exception ignored in: <function tqdm.__del__ at 0x7ff6bbb70f80>
Traceback (most recent call last):
  File

AttributeError: 'MCMC' object has no attribute 'summary'

### Exercise 2

A bivariate Gibbs sampler for a vector $x=(x_1,x_2)$ draws iteratively from the posterior conditional distributions in the following way:
- choose a starting value $p(x_1|x_2^{(0)})$
- for each iteration $i$:
    - draw $x_2(i)$ from $p(x_2|x_1^{(i-1)})$
    - draw $x_1(i)$ from $p(x_1|x_2^{(i)})$

a. Supposing that samples are drawn from a bivariate normal distribution

$$
{x_1 \choose x_2} \sim \mathcal{N} \Bigg[ {0 \choose 0} , \begin{pmatrix} 1 & \rho \\ \rho & 1 \end{pmatrix} \Bigg],
$$
    implement a Gibbs sampler for $x$ which takes as inputs the number of iterations `iters` and the number of warmup draws `warmup`.

b. Use your implementation of Gibbs sampler to infer the parameters $\theta=(\theta_1,\theta_2)$ from **Exercise 1**.